In [1]:
! pip install flash-attn tiktoken einops triton python-dotenv pytest

In [2]:
import json
import os
from dotenv import load_dotenv
from minsearch import Index

load_dotenv()

os.environ["HF_HOME"] = "/run/cache/"

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)
model_id = "microsoft/Phi-3-small-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype="auto",
  
    
    trust_remote_code=True
)
assert torch.cuda.is_available(), "This model needs a GPU to run ..."
device = torch.cuda.current_device()
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id,  trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
print(torch.cuda.is_available(), torch.cuda.current_device())

True 0


In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device
)
# not generation_args:
generation_args = {
        "max_new_tokens": 500,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False
    }

output = pipe("How to get rid of errors in kernel for jupyter notebook", **generation_args)

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/run/cache/modules/transformers_modules/microsoft/Phi-3-small-128k-instruct/f80aaa30bfc64c2b8ab214b541d9050e97163bc4/triton_flash_blocksparse_attn.py:88: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  x = [xi.to_sparse_csr() for xi in x]


In [8]:
documents = []

with open('documents.json', 'r') as file:

    docs = json.load(file)
    
    

In [9]:
for doc in docs:

    for document in doc['documents']:

        document['course'] = doc['course'] 
        documents.append(document)

In [10]:
textfields = ["text", "section", "question"]


indobject = Index(text_fields = textfields, keyword_fields = ['course'])
indobject.fit(documents)

In [11]:
def search(query, boost_dict = {"question": 3}, filter_dict =  {"course":"mlops-zoomcamp"}, num_results = 5 ):

    context = indobject.search(query = query, boost_dict = boost_dict, filter_dict = filter_dict, num_results = num_results
                    )
    
    return context

In [12]:
def build_prompt(query, related_docs):
    
    prompt = """
    You are an assistant for teaching online courses answer the question based on the context provided.
    Only the information available in the provided context should be used strictly adhere to this
    
    question:{query}
    
    context:{context}
    
    
    """.strip()
    
    
    context = ""
    for doc in related_docs:
        
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']} \n\n"
        
    
    prompt = prompt.format(query = query, context = context).strip()
    
    return prompt

In [13]:
def chat(prompt, generation_args = {}):
    
    messages = [{"role":"user", "content": prompt.strip()}]
    pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device
)
    if not generation_args:
        generation_args = {
        "max_new_tokens": 500,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False
    }

    output = pipe(messages, **generation_args)
    print(output)
    
    return output[0]['generated_text']
    
    
    
    

In [14]:
def rag(query, boost_dict = {"question": 3}, course_filter =  {"course": "data-engineering-zoomcamp"}, generation_args ={}):
    
    context = search(query = query, boost_dict = boost_dict, filter_dict = course_filter)
    
    prompt = build_prompt(query, context)
    
    answer = chat(prompt, generation_args = generation_args)
    
    return answer
    

In [15]:
query = "How to run spark engine ?"

In [ ]:
answer = rag(query)

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/run/cache/modules/transformers_modules/microsoft/Phi-3-small-128k-instruct/f80aaa30bfc64c2b8ab214b541d9050e97163bc4/triton_flash_blocksparse_attn.py:88: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  x = [xi.to_sparse_csr() for xi in x]
